In [3]:
import cv2
import numpy as np
from __future__ import division
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist
import winsound


def resize(img, width=None, height=None, interpolation=cv2.INTER_AREA):
    global ratio
    w, h = img.shape
    if width is None and height is None:
        return img
    elif width is None:
        ratio = height / h
        width = int(w * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized
    else:
        ratio = width / w
        height = int(h * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized

def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(36,48):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

camera = cv2.VideoCapture(0)

predictor_path = 'E:\\shape_predictor_68_face_landmarks.dat_2'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
while True:
    ret, f = camera.read()
    frame = cv2.flip(f, 1)

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_resized = resize(frame_gray, width=120)

# Upsample the image one time to make everything bigger

    dets = detector(frame_resized, 1)
    
    if len(dets) > 0:
        for k, d in enumerate(dets):
            shape = predictor(frame_resized, d)
            shape = shape_to_np(shape)
            leftEye= shape[lStart:lEnd]
            rightEye= shape[rStart:rEnd]
            leftEAR= eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            
            if ear>.29:
                m=1
                global count
                count = 0
                cv2.putText(frame, "Eyes Open ", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 10, 255), 2)
            else:  
                m=0
                #global count
                count = count + 1
                
# If the person sleeps while driving, the following condition will detect it 

                if(count > 30):
                    cv2.putText(frame, "Person Sleeping!!", (50,150),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            
# It produces a beep sound at 3500 Hz
                    winsound.Beep(3500, 500)
                
                cv2.putText(frame, "Eyes Blink", (10,30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100,10, 255), 2)
            
# Draw the keypoints around the eyes and track it
    
            for (x, y) in shape:
                cv2.circle(frame, (int(x/ratio), int(y/ratio)), 1, (255, 255, 255), 1)
    cv2.imshow("image", frame)

# Exits the program when you press q

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        camera.release()
        break